In [ ]:
# initialization for my classroom
import os
from datetime import datetime as dt

def logfile(user=os.environ.get('JUPYTERHUB_USER') or 'jovyan'):
    prefix='/srv'
    if os.path.isdir(prefix) and os.access(prefix, os.W_OK):
        prefix+=('/'+user)
        if not os.path.isdir(prefix):
            os.makedirs(prefix)
    else:
        prefix='.'
    return prefix+'/'+dt.now().strftime('%Y%m%d')+'.log'

path=logfile()
#%logstop
%logstart -otq $path append

# [python - cannot override sys.excepthook - Stack Overflow](https://stackoverflow.com/questions/1261668/cannot-override-sys-excepthook/28758396)
# https://github.com/ipython/ipython/blob/e6432249582e05f438303ce73d082a0351bb383e/IPython/core/interactiveshell.py#L1952

import sys
import traceback
import IPython

try:
    _showtraceback
except NameError:
    _showtraceback=IPython.core.interactiveshell.InteractiveShell.showtraceback

try:
    _showsyntaxerror
except NameError:
    _showsyntaxerror=IPython.core.interactiveshell.InteractiveShell.showsyntaxerror

import logging
logging.basicConfig(filename=path.replace('.log','-exc.log'), format='%(asctime)s %(message)s', level=logging.ERROR, force=True)

import sys
import traceback
import IPython

def showtraceback(self, *args, **kwargs):
    etype, value, tb = self._get_exc_info(kwargs.get('exc_tuple'))
    stb = self.InteractiveTB.structured_traceback(
        etype, value, tb, tb_offset=kwargs.get('tb_offset'))
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showtraceback(self, *args, **kwargs)

def showsyntaxerror(self, *args, **kwargs):
    etype, value, last_traceback = self._get_exc_info()
    elist = traceback.extract_tb(last_traceback) if kwargs.get('running_compiled_code') else []
    stb = self.SyntaxTB.structured_traceback(etype, value, elist)
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showsyntaxerror(self, *args, **kwargs)

IPython.core.interactiveshell.InteractiveShell.showtraceback = showtraceback
IPython.core.interactiveshell.InteractiveShell.showsyntaxerror = showsyntaxerror

# 課題-1) 基数の素因数の積で表される数を列挙する

* [Radix - Wikipedia](https://en.wikipedia.org/wiki/Radix)
* [Integer factorization - Wikipedia](https://en.wikipedia.org/wiki/Integer_factorization)
    * [Integer factorization in python - Stack Overflow](https://stackoverflow.com/questions/32871539/integer-factorization-in-python)
        * [Pollard's rho algorithm - Wikipedia](https://en.wikipedia.org/wiki/Pollard%27s_rho_algorithm)

## 因数を求める (素因数ではない)

In [ ]:
# https://stackoverflow.com/questions/32871539/integer-factorization-in-python

import math

def factorization(n):

    factors = []

    def get_factor(n):
        x_fixed = 2
        cycle_size = 2
        x = 2
        factor = 1

        while factor == 1:
            for count in range(cycle_size):
                if factor > 1: break
                x = (x * x + 1) % n
                factor = math.gcd(x - x_fixed, n)

            cycle_size *= 2
            x_fixed = x

        return factor

    while n > 1:
        next = get_factor(n)
        factors.append(next)
        n //= next

    return factors

In [ ]:
factorization(10), factorization(60)

In [ ]:
import numpy as np

np.prod(factorization(10)), np.prod(factorization(60))

## 素数2, 3, 5を因数に持つ数を列挙する

次の式で表される数を列挙する:

$$2^i \times 3^j \times 5^k$$

In [ ]:
for i in range(2):
    for j in range(2):
        for k in range(2):
            print(i,j,k)

In [ ]:
for i in range(2):
    for j in range(2):
        for k in range(2):
            print(2**i, 3**j, 5**k)

In [ ]:
for i in range(2):
    for j in range(2):
        for k in range(2):
            print(2**i * 3**j * 5**k)

In [ ]:
l = []
for i in range(2):
    for j in range(2):
        for k in range(2):
            l.append(2**i * 3**j * 5**k)
l

In [ ]:
sorted(l)

In [ ]:
sorted(l, reverse=True)

課題-1) $2^{i} \times 3^{j} \times 5^{k}$ で表される数を列挙する関数 `generate_p` を定義せよ:
  - 引数 `i_max`, `j_max`, `k_max` を取り、デフォルト値はそれぞれ `2` とする
  - 整数 $0 \leq $ `i` $\lt$ `i_max`, $0 \leq $ `j` $\lt$ `j_max`, $0 \leq $ `k` $\lt$ `k_max` の全ての組み合わせを列挙する
  - 昇順にソートされた整数のリストを戻り値とする

In [ ]:
def generate_p(i_max=2, j_max=2, k_max=2):
    # YOUR CODE HERE
    raise NotImplementedError()

## 冗長な記述を短くする

### `for` ループ

整数 $0 \leq $ `i` $\lt$ `i_max`, $0 \leq $ `j` $\lt$ `j_max`, $0 \leq $ `k` $\lt$ `k_max` の全ての組み合わせを列挙する:

* `itertools.product()`
  - [itertools — Functions creating iterators for efficient looping — Python 3.10.4 documentation](https://docs.python.org/3/library/itertools.html#itertools.product)

In [ ]:
import itertools

itertools.product(range(2),range(2),range(2))

In [ ]:
list(itertools.product(range(2),range(2),range(2)))

In [ ]:
for ijk in itertools.product(range(2),range(2),range(2)):
    i,j,k = ijk
    print(2**i * 3**j * 5**k)

### 掛け算

* `zip()`
  - [組み込み関数 — Python 3.10.4 ドキュメント](https://docs.python.org/ja/3/library/functions.html#zip)
* `np.prod()`
  - [numpy.prod — NumPy v1.22 Manual](https://numpy.org/doc/stable/reference/generated/numpy.prod.html)

In [ ]:
zip([1,2,3], [3,2,1])

In [ ]:
list(zip([1,2,3], [3,2,1]))

* `zip()` は転置行列を作る
  - [python matrix transpose and zip - Stack Overflow](https://stackoverflow.com/questions/10169919/python-matrix-transpose-and-zip)

In [ ]:
m = [[1,2,3],[3,2,1]]

In [ ]:
list(zip(*m))

In [ ]:
for ijk in itertools.product(range(2),range(2),range(2)):
    print(list(((2,3,5), ijk)))

In [ ]:
for ijk in itertools.product(range(2),range(2),range(2)):
    print(list(zip((2,3,5), ijk)))

In [ ]:
for ijk in itertools.product(range(2),range(2),range(2)):
    for n,m in zip((2,3,5), ijk):
        print(n**m, end=", ")
    print()

In [ ]:
for ijk in itertools.product(range(2),range(2),range(2)):
    print([n**m for n,m in zip((2,3,5), ijk)])

In [ ]:
for ijk in itertools.product(range(2),range(2),range(2)):
    print(np.prod([n**m for n,m in zip((2,3,5), ijk)]))

In [ ]:
[np.prod([n**m for n,m in zip((2,3,5), ijk)]) for ijk in itertools.product(range(2),range(2),range(2))]

## 補足

In [ ]:
i = itertools.product(range(2),range(2),range(2))
i

In [ ]:
next(i)

In [ ]:
while True:
    try:
        print(next(i))
    except StopIteration:
        break

In [ ]:
next(i)

---
以下、採点用のセルにつき編集できない:

In [ ]:
from nose.tools import assert_equal, assert_true

In [ ]:
assert_equal(generate_p(), [1, 2, 3, 5, 6, 10, 15, 30])

In [ ]:
assert_equal(generate_p(4,3,2), [1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 15, 18, 20, 24, 30, 36, 40, 45, 60, 72, 90, 120, 180, 360])